# Hitag-2 Brute Force CPU naive version

Author: [Kaci Amaouche](mailto:amaouchekaci28@gmail.com)

In this [Jupyter](https://jupyter.org/) notebook, we introduce the principle of the Guess and Determine attack, followed by a brute-force attack on this system, and explore each possible optimization. If you are unfamiliar with Jupyter, you can take a quick look at the [Notebook Basics](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Notebook%20Basics.html) guide (~5min).

Here is an outline of the session:

* [Environment Setup]
* [Guess and Determine attack]
* [Implementation and Explanation]

## 1 - Environment Setup

To run this lab on your own laptop you will need:
1. This notebook (the `.ipynb` file)
1. Python >= 3.8
2. Only native Python libraries were used.

## 2 - Guess and Determine attack

The idea of the attack relies on reversing the non-linear boolean function f, which takes 5 bits as input, and these 5 bits are outputs of functions that, in turn, take 4 bits as input.
The goal of the attack is not to directly recover the secret key K, but to retrieve the internal state of the register. In this case, we can find the key using the following theorem.

\{Definition 3.5.} Given a key $k = k_0 ... k_{47} \in \mathbb{F}_{2^{48}}$, an identifier $id = id_0 ... id_{31} \in \mathbb{F}_{2^{32}}$, a reader nonce $nR = nR_0 ... nR_{31} \in \mathbb{F}_{2^{32}}$, a reader answer $aR = aR_0 ... aR_{31} \in \mathbb{F}_{2^{32}}$, and a transponder answer $aT = aT_0 ... aT_{31} \in \mathbb{F}_{2^{32}}$, the internal state of the cipher at time $i$ is $\alpha_i := a_i ... a_{47+i} \in \mathbb{F}_{2^{48}}$. Here, the $a_i \in \mathbb{F}_2$ are given by:
\begin{align*}
a_i &:= id_i \quad \text{for } i \in [0,31] \\
a_{32+i} &:= k_i \quad \text{for } i \in [0,15] \\
a_{48+i} &:= k_{16+i} \oplus nR_i \quad \text{for } i \in [0,31] \\
a_{80+i} &:= L(a_{32+i} ... a_{79+i}) \quad \text{for } i \in \mathbb{N}
\end{align*}

Furthermore, we define the keystream bit $b_i \in \mathbb{F}_2$ at time $i$ by $b_i := f(a_i ... a_{47+i})$ for $i \in \mathbb{N}$.

Define $\{nR\}, \{aR\}_i, \{aT\}_i \in \mathbb{F}_2$ by:
\begin{align*}
\{nR\}_i &:= nR_i \oplus b_i \quad \text{for } i \in [0,31] \\
\{aR\}_i &:= aR_i \oplus b_{32+i} \quad \text{for } i \in [0,31] \\
\{aT\}_i &:= aT_i \oplus b_{64+i} \quad \text{for } i \in [0,31]
\end{align*}

Note that the $a_i, \alpha_i, b_i, \{nR\}_i, \{aR\}_i,$ and $\{aT\}_i$ are formally functions of $k$, $id$, and $nR$. Instead of making this explicit by writing, for example, $a_i(k,id,nR)$, we just write $a_i$ where $k$, $id$, and $nR$ are clear from the context.

{3.7 Rollback}

To recover the key, it is sufficient to learn the internal state of the cipher $\alpha_i$ at any point $i$ in time. Since an attacker knows $id$ and $\{nR\}$, the LFSR can then be rolled back to time zero.

{Definition 3.6.} The rollback function $R: \mathbb{F}_{2^{48}} \rightarrow \mathbb{F}_2$ is defined by $R(x_1 ... x_{48}) := x_2 \oplus x_3 \oplus x_6 \oplus x_7 \oplus x_8 \oplus x_{16} \oplus x_{22} \oplus x_{23} \oplus x_{26} \oplus x_{30} \oplus x_{41} \oplus x_{42} \oplus x_{43} \oplus x_{46} \oplus x_{47} \oplus x_{48}$.

If one first shifts the LFSR left using $L$ to generate a new bit on the right, then $R$ recovers the bit that dropped out on the left, i.e.,
$R(x_1 ... x_{47} L(x_0 ... x_{47})) = x_0$. (1)

{Theorem 3.7.} In the situation from Definition 3.5, we have:
\begin{align*}
a_{32+i} &= R(a_{33+i} ... a_{80+i}) \quad \text{for } i \in \mathbb{N} \\
a_i &= id_i \quad \text{for } i \in [0,31]
\end{align*}

{Proof.} Straightforward, using Definition 3.5 and Equation (1).

If an attacker manages to recover the internal state of the LFSR $\alpha_i = a_ia_{i+1} ... a_{i+47}$ at some time $i$, then she can repeatedly apply Theorem 3.7 to recover $a_0a_1 ... a_{79}$ and, consequently, the keystream $b_0b_1b_2 ...$. By having eavesdropped $\{nR\}$ from the authentication protocol, the adversary can further calculate:
$nR_i = \{nR\}_i \oplus b_i \quad \text{for } i \in [0,31]$

Finally, the adversary can compute the secret key as follows:
\begin{align*}
k_i &= a_{32+i} \quad \text{for } i \in [0,15] \\
k_{16+i} &= a_{48+i} \oplus nR_i \quad \text{for } i \in [0,31]
\end{align*}


Assuming that the attacker has intercepted a sequence ks0k1....ks31, their objective is to guess the bits that produce each bit ks(i). They only require the first 9 bits ks0, ks1...ks8. In accordance with the Hitag-2 system scheme, we can express ks0 as a function of the following 20 input bits: ks0 = f(a2,a3,a5,a6,a8,a12,a14,a15,a17,a21,a23,a26,a28,a29,a31,a33,a34,a43,a44,a46).

These 20 bits serve as inputs to the sub-functions fb and fa. We can represent these bits using masks. For example, if we need to guess (a0, a1, a5), we utilize the mask ('110001'), where a 1 denotes a bit to be guessed.

For ks0, there are 2^20 candidates capable of producing this bit. However, half of the candidates can be eliminated in this phase due to the fact that f is a balanced function.

Next, for ks1, the register is shifted to the left, and we can express it as:
ks1 = f(a3, a4,a6,a7,a9,a13,a15,a16,a18,a22,a24,a27,a29,a30,a32,a34,a35,a44,a45,a47). Considering that a3, a6, a15, a29, a34, and a44 have already been guessed, there are only 14 bits to be determined instead of 20.

At this stage, the attacker would have 2^19 * 2^14 = 2^33 candidates. Then, they eliminate half of the candidates, resulting in 2^32 candidates.

Following the same logic, for ks2, there are only 4 bits to be determined, leading to (2^32 * 2^4)/2 = 2^35 candidates.

For ks3, there are only 3 bits to be determined, resulting in (2^35 * 2^3)/2 = 2^37 candidates.

From ks3 to ks8, there is only one bit to be determined in each step (layer). This algorithm will generate approximately 2^39 possible states of the LFSR that have generated the sequence ks0...ks8. Therefore, it is sufficient to test these 2^39 states against the remaining 23 bits ks9...ks31, which will produce a list of 2^16 candidate states.


With a second triplet (uid, iv2, ks2), it is possible to reduce this list to a single candidate state.

## 3 - Implementation and Explanation

This first implementation is a brute force attack (without using guess and determine), targeting the secret key directly rather than the internal state of the register.
It takes approximately 194 days in the worst case scenario on a standard PC.
Note that this execution time refers to a C implementation, while in our example we use a Python implementation for simplicity.

First, we begin by defining the Hitag-2 cryptosystem. To do so, we first define the 5 subfunctions and the nonlinear function fc as follows: (each function is explained in detail in the documentation)

In [20]:
#An optimized method for computing the non-linear function fc.
def i4(x, a, b, c, d):
    """
    Computes a value by extracting specific bits from the input 'x' based on the given indices 'a', 'b', 'c', and 'd'.

    Parameters:
    x (int): The input value.
    a, b, c, d (int): Indices specifying the position of the bits to extract.

    Returns:
    int: The computed value based on the extracted bits.
    """
    return (((x >> a) & 1)*8)+((x >> b) & 1)*4+((x >> c) & 1)*2+((x >> d) & 1)


def f20_4(state):
    """
    Computes the value of the fourth component of the 'f20' function based on the given 'state'.

    Parameters:
    state (int): The input state.

    Returns:
    int: The computed value of the fourth component of 'f20'.
    """
    return ((0x3c65 >> i4(state,34,43,44,46)) & 1)

def f20_3(state):
    """
    Computes the value of the third component of the 'f20' function based on the given 'state'.

    Parameters:
    state (int): The input state.

    Returns:
    int: The computed value of the third component of 'f20'.
    """
    return (( 0xee5 >> i4(state,28,29,31,33)) & 1)

def f20_2(state):
    """
    Computes the value of the second component of the 'f20' function based on the given 'state'.

    Parameters:
    state (int): The input state.

    Returns:
    int: The computed value of the second component of 'f20'.
    """
    return (( 0xee5 >> i4(state,17,21,23,26)) & 1)

def f20_1(state):
    """
    Computes the value of the first component of the 'f20' function based on the given 'state'.

    Parameters:
    state (int): The input state.

    Returns:
    int: The computed value of the first component of 'f20'.
    """
    return (( 0xee5 >> i4(state, 8,12,14,15)) & 1)

def f20_0(state):
    """
    Computes the value of the zeroth component of the 'f20' function based on the given 'state'.

    Parameters:
    state (int): The input state.

    Returns:
    int: The computed value of the zeroth component of 'f20'.
    """
    return ((0x3c65 >> i4(state, 2, 3, 5, 6)) & 1)

def f20_last(s0,s1,s2,s3,s4):
    """
    Computes the last component of the 'f20' function based on the given five input components.

    Parameters:
    s0, s1, s2, s3, s4 (int): The five input components.

    Returns:
    int: The computed value of the last component of 'f20'.
    """
    return (0xdd3929b >> ((s0 * 16)
                        + (s1 *  8)
                        + (s2 *  4)
                        + (s3 *  2)
                        + (s4 *  1))) & 1

#The fc function
def f20(state):
    """
    Computes the final value of the 'f20' function based on the given 'state'.

    Parameters:
    state (int): The input state.

    Returns:
    int: The computed value of the 'f20' function.
    """
    return f20_last(f20_0(state), f20_1(state), f20_2(state), f20_3(state), f20_4(state))



Now, we define an algorithm which Performs the initialization phase of the Hitag-2 algorithm

In [21]:
#The initialization phase of Hitag-2 
def hitag2_init(key, uid, nonce):
    """
    Performs the initialization phase of the Hitag-2 algorithm.

    Parameters:
    key (int): The key value.
    uid (int): The unique identifier value.
    nonce (int): The nonce value.

    Returns:
    int: The resulting state after the initialization phase.
    """
    state = 0

    # Extract the key bits and append them to the state
    for i in range(32, 48):
        state = (state << 1) | ((key >> i) & 1)

    # Extract the UID bits and append them to the state
    for i in range(0, 32):
        state = (state << 1) | ((uid >> i) & 1)

    # Generate the state based on the nonce bits and key bits
    for i in range(0, 32):
        nonce_bit = (f20(state) ^ ((nonce >> (31-i)) & 1))
        state = (state >> 1) | (((nonce_bit ^ (key >> (31-i))) & 1) << 47)
    return state

And now we define the lfsr_feedback and the new state of the register after one clock cycle.

In [22]:
def lfsr_feedback(state):
    """
    Computes the feedback bit for the LFSR (Linear Feedback Shift Register) based on the given state.

    Parameters:
    state (int): The current state of the LFSR.

    Returns:
    int: The computed feedback bit.
    """
    return (((state >>  0) ^ (state >>  2) ^ (state >>  3)
            ^ (state >>  6) ^ (state >>  7) ^ (state >>  8)
            ^ (state >> 16) ^ (state >> 22) ^ (state >> 23)
            ^ (state >> 26) ^ (state >> 30) ^ (state >> 41)
            ^ (state >> 42) ^ (state >> 43) ^ (state >> 46)
            ^ (state >> 47)) & 1)

def lfsr(state):
    """
    Updates the state of the LFSR (Linear Feedback Shift Register) based on the given state.

    Parameters:
    state (int): The current state of the LFSR.

    Returns:
    int: The updated state of the LFSR.
    """
    return (state >>  1) + (lfsr_feedback(state) << 47)


For each such state that has generated 32 bits of
keystream output the LFSR function can then be applied
in reverse 32 times to get its initial state.

And for that, we define a function that reverses the register to allow us to go back in time

In [23]:
def lfsr_feedback_inv(state):
    """
    Computes the inverse feedback bit for the LFSR (Linear Feedback Shift Register) based on the given state.

    Parameters:
    state (int): The current state of the LFSR.

    Returns:
    int: The computed inverse feedback bit.
    """
    return (((state >>  47) ^ (state >>  1) ^ (state >>  2)
            ^ (state >>  5) ^ (state >>  6) ^ (state >>  7)
            ^ (state >> 15) ^ (state >> 21) ^ (state >> 22)
            ^ (state >> 25) ^ (state >> 29) ^ (state >> 40)
            ^ (state >> 41) ^ (state >> 42) ^ (state >> 45)
            ^ (state >> 46)) & 1)

def lfsr_inv(state):
    """
    Updates the inverse state of the LFSR (Linear Feedback Shift Register) based on the given state.

    Parameters:
    state (int): The current state of the LFSR.

    Returns:
    int: The updated inverse state of the LFSR.
    """
    return ((state <<  1) + (lfsr_feedback_inv(state))) & ((1<<48)-1)

We can verify that for any x, we always have lfsr(lfsr_inv(x)) == x.

In [28]:
x=52
print(lfsr_inv(lfsr(x))==x)

True


Now we need to implement the algorithm that computes the 32 bits of the keystream.

In [29]:
#This returns the first 32 bits of the keystream
def hitag2(state,KEYSIZE=32):
    """
    Generates the first 32 bits of the keystream using the Hitag-2 algorithm.

    Parameters:
    state (int): The current state of the Hitag-2 algorithm.
    KEYSIZE (int): The size of the keystream to generate (default: 32).

    Returns:
    int: The first 32 bits of the keystream.
    """
    keystream = 0
    for _ in range(0, 32):
        # Append the output bit of 'f20' to the rightmost bit of the keystream
        keystream = (keystream << 1) | f20(state)

        # Update the state using the LFSR function
        state = lfsr(state)
    return keystream

To compute the keystream associated with the key, UID, and IV, we can proceed as follows:

In [ ]:
key, uid, iv = 0x414141414141, 0x42424242, 0x43434343
state = hitag2_init(key, uid, iv)
keystream=hitag2(state)
print(keystream)

Now that we have all the necessary ingredients, we can implement a brute force attack that directly targets the secret key K by iteratively testing all possible combinations.

Once a candidate key is found, we test it against the second vector IV2, and if it produces the same keystream KS2, then it is the correct key. If not, we continue with the next candidate.

In [31]:
def BruteForce(uid,iv1,iv2,ks1,ks2):
    """

    This function implements a naive brute force attack on the 48-bit key K.
    param : It takes a serial number (uid) and two sets of (iv,ks) pairs as arguments
    Output : The secret key K
    
    """

    #We iterate over all possible candidate keys, which are 2^48 in total.
    for key_cand in range(1<<48):
        #We calculate the state of the register using the candidate key
        state=hitag2_init(key_cand,uid,iv1)

        #We call the Hitag2 function with this state as an argument. If it produces the correct keystream, we add the candidate key to the list of candidates
        if hitag2(state,32)==ks1:
            state2=hitag2_init(key_cand,uid,iv2)
            #Since the keystream is 32 bits, there are on average 2^48 - 2^32 = 2^16 keys that can produce the same (id, iv, ks) triplet due to possible collisions.
            #The second triplet helps to resolve this ambiguity and will return only one possible key.
            if hitag2(state2,32)==ks2:
                return key_cand

We can verify the validity of this attack as follows: (Note that we choose a relatively small value for the secret key to avoid long waiting times)

We define the key, uid, iv1, and iv2, and then we compute the corresponding ks1 and ks2.

In [33]:
key, uid, iv1, iv2 = 2**14-1, 0x42424242, 0x43434343,0x43434344
state1 = hitag2_init(key, uid, iv1)
state2 = hitag2_init(key, uid, iv2)
ks1, ks2 = hitag2(state1,32), hitag2(state2,32)

We verify that the key returned by the algorithm is the same as the one we used.

In [36]:
print(BruteForce(uid, iv1, iv2, ks1, ks2)==key)

True
